In [ ]:
from ollama import Client
import os
from dotenv import load_dotenv
from mitreattack.stix20 import MitreAttackData

# Load environment variables from .env file
load_dotenv()

In [3]:
client = Client(host=os.getenv("OLLAMA_HOST"))
mitre_attack_data = MitreAttackData("enterprise-attack.json")

In [6]:
# Extract keywords from the description
def extract_keywords_from_description(description):
    # Define the merged prompt
    prompt = (f"Given the cybersecurity scenario description: '{description}', identify and list the key terms, "
              "techniques, or technologies relevant to MITRE ATT&CK. Extract TTPs from the scenario. "
              "If the description is too basic, expand upon it with additional details, applicable campaign, "
              "or attack types based on dataset knowledge. Then, extract the TTPs from the revised description.")
    # Set up the messages for the Ollama API
    messages = [
        {
            "role": "system",
            "content": "You are a cybersecurity professional with more than 25 years of experience."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    # Set the request options
    options = {
        "temperature": 0.7
    }
    # Make the API call
    try:
        client = Client(host=os.getenv("OLLAMA_HOST"))
        response = client.chat(
            model="llama3",
            messages=messages,
            options=options,
            stream=False            
        )
        response_content = response['message']['content'].strip()
        keywords = response_content.split(', ')
        return keywords
    except Exception as e:
        print("An error occurred while connecting to the OpenAI API:", e)
        return []

In [11]:
# Generate a comprehensive scenario
def generate_ttp_chain(match):
    # Create a prompt for GPT-3 to generate a TTP chain for the provided match
    prompt = (f"Given the MITRE ATT&CK technique '{match['name']}' and its description '{match['description']}', "
              "generate an example scenario and TTP chain demonstrating its use.")
    # Set up the messages for the OpenAI API
    messages = [
        {
            "role": "system",
            "content": "You are a cybersecurity professional with expertise in MITRE ATT&CK techniques."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    # Set the request options
    options = {
        "temperature": 0.7
    }
    # Make the API call
    try:
        client = Client(host=os.getenv("OLLAMA_HOST"))
        response = client.chat(
            model="llama3",
            messages=messages,
            options=options,
            stream=False            
        )
        response_content = response['message']['content'].strip()
        return response_content
    except Exception as e:
        print("An error occurred while generating the TTP chain:", e)
        return "Unable to generate TTP chain."

In [9]:
# Search the ATT&Ck dataset for extracted keywords
def score_matches(matches, keywords):
    scores = []
    for match in matches:
        score = sum([keyword in match['name'] for keyword in keywords]) + \
                sum([keyword in match['description'] for keyword in keywords])
        scores.append((match, score))
    return scores
def search_dataset_for_matches(keywords):
    matches = []
    for item in mitre_attack_data.get_techniques():
        if any(keyword in item['name'] for keyword in keywords):
            matches.append(item)
        elif 'description' in item and any(keyword in item['description'] for keyword in keywords):
            matches.append(item)
    return matches

In [10]:
#description = input("Enter your scenario description: ")
description = "The scenario is this: an electic utility is being attacked by APT29."
keywords = extract_keywords_from_description(description)
matches = search_dataset_for_matches(keywords)
scored_matches = score_matches(matches, keywords)
# Sort by score in descending order and take the top 3
top_matches = sorted(scored_matches, key=lambda x: x[1], reverse=True)[:3]

In [12]:
print("Writing top 3 matches from the MITRE ATT&CK dataset:")
with open("ollama_red_team_scenarios.txt", "w") as file:
    for match, score in top_matches:
        file.write("Name:" + match['name'] +"\n")
        file.write("Summary:" + match['description'] + "\n")
        ttp_chain = generate_ttp_chain(match)
        file.write("Example Scenario and TTP Chain:" + ttp_chain + "\n")
        file.write("-" * 50 + "\n")

Writing top 3 matches from the MITRE ATT&CK dataset:
